# Investigating Crop Quality in Beaune, France with Remote Sensing instruments for the 2022 vintage.

The goal of this project is to investigate the tools freely available to the public in monitoring the health of *Vitis Vinifera* grapes in the vineyard. This study will focus on the Burgundy region as it produces some of my favorite wines using Pinot Noir.

Here I investigate Landsat 9 imagery by interfacing with [Google Earth Engine](https://earthengine.google.com/).
I will calculate NDVI and a handful of other vegitation indexes to see which are useful and compare the freely available products to private options which may be better suited.

Particular inspiration for this analysis came from [this study](https://www.frontiersin.org/articles/10.3389/fpls.2021.683078/full) conducted on a vineyard in Greece which concluded that *"both satellite-based and proximal-based NDVI at both stages (veraison and harvest) presented good correlations to crop quality characteristics ..."*. While this seems promising to my preliminary research it should be noted that the proximal based (in this study's case, drones within the vineyards) remote sensing applications were preffered to the remote sensing based (in this case Sentinel-2 archived imagery) instruments.

There are private companies like [VineView](https://vineview.com/data-products/vine-vigor-products/) and [VineScapes](https://www.vinescapes.com/read-our-published-work/) already using remote sensing techniques to offer insight into vineyard management.

In [ ]:
# Uncomment these if you do not have the dependencies installed

# !pip install ee
# !pip install geemap

import ee
import geemap

In [ ]:
# Get our geemaps map set up
burgundy_map = geemap.Map(center = ( 47.02882844813948, 4.948176654205355), zoom = 11, layer_ctrl = True, data_ctrl = False, add_google_map = False)


Now let's grab our data.

I was able to find polygons representing the region of Burgundy in geojson format [here](https://github.com/gregoiredavid/france-geojson/tree/master/regions/bourgogne-franche-comte).

Let's save the time frame we are interested in as a variable so we can ensure all our datasets are all representing the same timeframe.

In [ ]:
study_area = ee.Geometry.Polygon([[4.739553014260849, 47.1586977719042],[4.739553014260849,46.90343743229144],[5.143402445712439,46.90343743229144],[5.143402445712439,47.1586977719042],[4.739553014260849,47.1586977719042]])
study_date = ('2022-08-01', '2022-12-30')

# burgundy_arrondisement = geemap.geojson_to_ee('https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/regions/bourgogne-franche-comte/arrondissements-bourgogne-franche-comte.geojson')

# Get landsat 8 imagery
image_2022 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2").filterDate(study_date[0], study_date[1]).filterBounds(study_area).sort('CLOUD_COVER').first().clip(study_area)


Let's investigate our photo to see what we've got after all that filtering and clipping.

In [ ]:
props = geemap.image_props(image_2022)

print(f'The clearest Landsat photo during the 2022 harvest season for our region was taken on {props.get("IMAGE_DATE").getInfo()} with a cloud cover of {(props.get("CLOUD_COVER_LAND").getInfo())}%')

### Lucky us!

This date should work out well for our analysis since the 2022 harvest in Burgundy began around August 25th. So we will get a look at the vines just a couple weeks before harvest! And on a day with very low cloud coverage!

The weeks prior to harvest are some of the most stressful for a winemaker. Too little or too much of rain, humidity, sunlight, or soil nutrients can dramatically impact the final quality of the wine by veering the grape from the desired PH and sugar levels. While this may not be a problem for more "modern" regions, Burgundy maintains that the role of the vintner in the final product should be minimal and it is their job to simply see through what the terroir has given them.<br>This approach to minimal intervention winemaking makes for more terroir driven wines but leaves the product at the mercy of nature.

Having a snapshot into these stressful times should be incredibly interesting!

### Landsat 9 Data Scaling

All Landsat data needs to be rescaled before any analysis or visualization can occus, you can reasd more on the topic [here](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC09_C02_T1_L2#bands).

In [ ]:
def applyScaleFactors(image: ee.Image) -> ee.Image:
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

# landsat_9_data_2022 = applyScaleFactors(landsat_9_image_2022)
landsat_8_data_2022 = applyScaleFactors(image_2022)

Let's save the visualization parameters for our different layers in dicts so we can visualize them more programatically.

In [ ]:
vis_natural = {
    'name': 'Natural Color (4,3,2)',
    'properties': {
        'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
        'min': 0.0,
        'max': 0.3,
    }
}

vis_IR = {
    'name': 'False Color (5,4,3)',
    'properties': {
        'bands': ['SR_B5', 'SR_B4', 'SR_B3'],
        'min': 0.0,
        'max': 0.5
    }
}

vis_veg = {
    'name': 'Vegatation Visualization (6,5,4)',
    'properties': {
        'bands': ['SR_B6', 'SR_B5', 'SR_B4'],
        'min': 0.0,
        'max': 0.7
    }
}

vis_ndvi = {
    'name': 'NDVI',
    'properties': {
        'min': 0.0,
        'max': 1.0,
        'palette': [
            'FFFFFF',
            'CE7E45',
            'DF923D',
            'F1B555',
            'FCD163',
            '99B718',
            '74A901',
            '66A000',
            '529400',
            '3E8601',
            '207401',
            '056201',
            '004C00',
            '023B01',
            '012E01',
            '011D01',
            '011301',
        ]
    }
}


### Initial Visualization 

Alright that was a lot.

Let's put that all on a map so we can see what we've been doing so far.<br>We'll put a natural color and an infrared layer to more easily visualize the greenery of the region. We'll call on our previously assigned visualization parameters to tell geemap how to draw the layers.

In [ ]:
burgundy_map.add_ee_layer(
    ee_object = landsat_8_data_2022,
    vis_params = vis_IR['properties'],
    name = f'2022 {vis_IR["name"]}'
)
burgundy_map.add_ee_layer(
    ee_object = landsat_8_data_2022,
    vis_params = vis_natural['properties'],
    name = f'2022 {vis_natural["name"]}'
)

burgundy_map

### That looks pretty good!

With Landsat 9's 30m resolution we can clearly see the city of Beaune with the sprawling vineyards and forests to the east.
This will be the first harvest that Landsat 9 imagery can be used for analysis since its launch on September 27 2021.

In our infrared visualization, we can see the large red patch to the right of the image is the [Forêt domaniale de Citeaux](https://www.onf.fr/vivre-la-foret/forets-de-france/%2B/19b6::foret-domaniale-de-citeaux.html?lang=fr) which is an oak forest producing staves for many of the oak barrels used for aging wines throughout the region. This is why you can see the plot like structure of the land in both of our layers. Interestingly in the IR layer you can distinguish which plots are more mature, and likely to be harvested next whereas in the natural color you can only distinguish larger segmentation of the forest. A fun example of remote sensing's practicality.

If you need spatial reference, you can unselect all our added layers to see an Open Street Map base layer.

### NDVI Calculation

Ndvi has [long been used](https://spinoff.nasa.gov/spinoff2003/er_2.html) in vineyards to generalize vine health.

Farmers use it to monitor crops and decide when is the optimal time to pick the fruit.

The calculation for NDVI is fairly straight forward and involves finding the difference between the NIR and Red bands of our image.
$$
    NDVI = \frac{(NIR - Red)}{(NIR + Red)}\
$$

Let's calculate the NDVI of our image

In [ ]:
NIR = image_2022.select('SR_B5')
red = image_2022.select('SR_B4')

# This is the line that computes NDVI
ndvi = NIR.subtract(red).divide(NIR.add(red)).rename('NDVI')

Let's visualize our NDVI layer.<br>All previous layers are still available to play around with.

In [ ]:
burgundy_map.add_ee_layer(
    ee_object = ndvi,
    name = vis_ndvi['name'],
    vis_params = vis_ndvi['properties']
)
burgundy_map

Awesome, we've got our NDVI, and visualization layers overlapping to get a better understanding of the region of Beaune at harvest time.

My next curiosity is to see how the NDVI changes year over year. Vintages in the wine world are the year in which the wine was grown and are incredibly deterministic to the wine's quality thanks to the notion of [terroir](https://winefolly.com/tips/terroir-definition-for-wine/).
Let's see how the NDVI Has changed in the Beaune region since 2013 (the year Landsat 8 became operational) and compare that to the region's wine scores.

### What about Sentinal Imagery?
That's all very interesting and Landsat 9 is awesome however, [this study](https://www.mdpi.com/2077-0472/11/5/457#) found that *"In medium-resolution imagery, Sentinel-2 excels in use over others, due to the ability to reveal spatial variability that can be used on a regional scale."*.<br>
So let's see what the same NDVI calculation will give us with Sentinel-2 data.

In [ ]:
sentinal_2022 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
    .filterDate(study_date[0], study_date[1])\
    .filterBounds(study_area)\
    .sort('HIGH_PROBA_CLOUDS_PERCENTAGE')\
    .first()\
    .clip(study_area)

# have to divide sentinal bands by 100000 to make them useable
# Note this is only for bands 1-12 but that is all we will use for this so a blanket divide works fine.
sentinal_2022_data = sentinal_2022.divide(10000)

In [ ]:
props2 = geemap.image_props(sentinal_2022)

print(f'The clearest Sentinal photo during the 2022 harvest season for our region was taken on {props2.get("IMAGE_DATE").getInfo()} with a cloud cover of {round(props2.get("HIGH_PROBA_CLOUDS_PERCENTAGE").getInfo(), 2)}%')

Lucky us our best Sentinal photo for the time period was just 3 days after our Landsat one!<br>

Let's visualize it.

In [ ]:
# Sentinal uses different band combinations, so we'll update our visualization params.
sentinal_nat = {
    'name': 'Natural Color (4,3,2)',
    'properties': {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0.0,
        'max': 0.3,
    }
}

sentinal_IR = {
    'name': 'IR Color (4,3,2)',
    'properties': {
        'bands': ['B8', 'B4', 'B3'],
        'min': 0.0,
        'max': 0.3,
    }
}


In [ ]:
sentinal_map = geemap.Map(center = ( 47.02882844813948, 4.948176654205355), zoom = 11, layer_ctrl = True, data_ctrl = False, add_google_map = False)

sentinal_map.add_ee_layer(
    sentinal_2022_data,
    vis_params = sentinal_nat['properties'],
    name = sentinal_nat['name'],
)

sentinal_map

I mean that is awesome isn't it?! Sentinal-2 offers 10m resolution on it's bands and while it is not going to get us into individual rows of vines it sure looks great.

In [ ]:
# Calculate  zonal statistics
# geemap.zonal_statistics_by_group(
#     in_value_raster = ndvi_2015,
#     in_zone_vector = burgundy_arrondisement,
#     statistics_type='PERCENTAGE',
# )